In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
from mpl_toolkits.mplot3d import Axes3D
import netCDF4
import numpy
import os
import pandas
from PIL import Image
import random
import readline
import scipy
from scipy import fft
from scipy import linalg
from scipy import stats
from scipy.stats import poisson, ttest_ind
import seaborn
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import normalize
from statsmodels.tsa.ar_model import AutoReg
import xarray as xr

In [2]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExTraTrack/rcp_files/'

In [3]:
# Open File
def Create_DF(File):
    Data = open(File, 'r')
    Rows = []
#
# Organize Data
    for Line in Data:
        Rows.append(Line.strip())
    Storm_Code = []
    Storm_List_Orig = []
    for i in range(len(Rows)):
        if Rows[i][0:5] == 'start':
            Code = Rows[i][41:45]
            Storm_List_Orig.append(Code)
        else:
            Storm_Code.append(Code)
    Array = numpy.zeros((13, len(Rows)-len(Storm_List_Orig)))
    Time = []
    k = -1
    for i in range(len(Rows)):
        if Rows[i][0:5] == 'start':
            k += 1
        else:
            l = len(Rows[i]) - 100
            Array[0][i-k-1] = float(Rows[i][0:6+l])
            Array[1][i-k-1] = float(Rows[i][9+l:14+l])
            Array[2][i-k-1] = float(Rows[i][17+l:24+l])
            Array[3][i-k-1] = float(Rows[i][26+l:31+l])
            Array[4][i-k-1] = float(Rows[i][34+l:41+l])
            Array[5][i-k-1] = float(Rows[i][44+l:51+l])
            Array[6][i-k-1] = float(Rows[i][54+l:61+l])
            Array[7][i-k-1] = float(Rows[i][64+l:71+l])
            Array[8][i-k-1] = float(Rows[i][74+l:81+l])
            Time.append(datetime.datetime(year=int(Rows[i][84+l:88+l]), month=int(Rows[i][90+l:92+l]), \
            day=int(Rows[i][94+l:96+l]), hour=int(Rows[i][98+l:100+l])))
#
# Create DataFrame to Store Data
    DF_All = pandas.DataFrame({"Orig Code": Storm_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "Dist(m)": Array[4], "Angle": Array[5], "B": Array[6], "VLT": Array[7], "VUT": Array[8], \
    "Orig Time(Z)": Time})
#
# Remove -999.0 Data Points
    DF = DF_All[(DF_All["B"] > -728) & (DF_All["VLT"] > -728) & (DF_All["VUT"] > -728)].reset_index()
    return (DF, Storm_List_Orig)

In [4]:
# Combine DataFrames
def Combine_DF(DF_A, DF_B, DF_C, Orig_List_A, Orig_List_B, Orig_List_C, Model):
    if Model == "Control":
        Year_Start = 1900
#        Year_Diff_A = Year_Start - 1985
#        Year_Diff_B = Year_Start - 1985 + 30
#        Year_Diff_C = Year_Start - 1985 + 60
    New_List_A = []
    New_List_B = []
    New_List_C = []
    New_Time = []
    New_Code = []
    ABC = []
    Orig_Combine = []
    New_Combine = []
# Dataset A
    Year = Year_Start
    Year_Diff_A = Year - DF_A["Orig Time(Z)"][0].year
    Count = 0
    for k in range(len(Orig_List_A)):
        Code, Year, Count = Update_Code(DF_A, Orig_List_A[k], Year_Diff_A, Year, Count)
        New_List_A.append(Code)
# Dataset B
    Year += 1
    Year_Diff_B = Year - DF_B["Orig Time(Z)"][0].year
    Count = 0
    for k in range(len(Orig_List_B)):
        Code, Year, Count = Update_Code(DF_B, Orig_List_B[k], Year_Diff_B, Year, Count)
        New_List_B.append(Code)
# Dataset C
    Year += 1
    Year_Diff_C = Year - DF_C["Orig Time(Z)"][0].year
    Count = 0
    for k in range(len(Orig_List_B)):
        Code, Year, Count = Update_Code(DF_C, Orig_List_C[k], Year_Diff_C, Year, Count)
        New_List_C.append(Code)
#
# Combine Into New Time and Codes List
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_A, Orig_List_A, New_List_A, Year_Diff_A, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "A")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_B, Orig_List_B, New_List_B, Year_Diff_B, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "B")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_C, Orig_List_C, New_List_C, Year_Diff_C, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "C")
#
# Concatenate Other Variables
    Vars = ["Lon", "Lat", "SLP(hPa)", "Winds(m/s)", "Dist(m)", "Angle", "B", "VLT", "VUT"]
    Total_Length = len(New_Code)
    Array = numpy.zeros((9,Total_Length))
    for n in range(len(Vars)):
        Array[n] = numpy.concatenate((DF_A[Vars[n]], DF_B[Vars[n]], DF_C[Vars[n]]))
#
# Create DataFrame to Store Data
    DF_Combine = pandas.DataFrame({"Code": New_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "B": Array[6], "VLT": Array[7], "VUT": Array[8], "Time(Z)": New_Time})
    Codes_DF = pandas.DataFrame({"ABC": ABC, "Orig Code": Orig_Combine, "New Code": New_Combine})
    Year_Diffs = numpy.array([Year_Diff_A, Year_Diff_B, Year_Diff_C])
    return (DF_Combine, Codes_DF, Year_Diffs)

In [5]:
def Update_Code(DF, Orig_Code, Year_Diff_A, Year, Count):
    DF_Storm = DF[DF["Orig Code"] == Orig_Code].reset_index()
    Time_Year = DF_Storm["Orig Time(Z)"][0].year + Year_Diff_A
    if Time_Year == Year:
        Count += 1
    else:
        Year += 1
        Count = 1
    if Count < 10:
        Code = str(Year)+"0"+str(Count)
    else:
        Code = str(Year)+str(Count)
    return (Code, Year, Count)

In [6]:
def Combine_Lists(DF, Orig_List, New_List, Year_Diff, New_Code, New_Time, ABC, Orig_Combine, New_Combine, Y):
    for i in range(len(DF)):
        for l in range(len(Orig_List)):
# Update Storm Code
            if DF["Orig Code"][i] == Orig_List[l]:
                New_Code.append(New_List[l])
# Update Year
        Orig_Time = DF["Orig Time(Z)"][i]
        New_Time.append(Update_Year(Orig_Time, Year_Diff))
    for m in range(len(Orig_List)):
        ABC.append(Y)
        Orig_Combine.append(Orig_List[m])
        New_Combine.append(New_List[m])
    return (New_Code, New_Time, ABC, Orig_Combine, New_Combine)

In [7]:
def Update_Year(Orig_Time, Year_Diff):
    Year_Update = Orig_Time.year + Year_Diff
# Febuary 29 Problems
    if Orig_Time.month == 2 and Orig_Time.day == 29:
        New_Time = Orig_Time.replace(year=Year_Update, day=28)
    else:
        New_Time = Orig_Time.replace(year=Year_Update)
    return (New_Time)

In [8]:
# Open File
Control_A_DF_Orig, Control_A_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.002_avg')
Control_DF, Control_Codes, Control_Year_Diffs = Combine_DF(Control_A_DF_Orig, Control_A_DF_Orig, Control_A_DF_Orig, \
Control_A_Storm_List_Orig, Control_A_Storm_List_Orig, Control_A_Storm_List_Orig, "Control")
Control_DF

,Code,Lon,Lat,SLP(hPa),Winds(m/s),B,VLT,VUT,Time(Z)
0,190001,-82.19,27.99,1015.45,13.9,4.71,37.04,-36.64,1900-06-19 12:00:00
1,190001,-82.26,27.79,1015.45,13.8,6.18,-12.97,-23.15,1900-06-19 18:00:00
2,190001,-82.33,27.58,1015.45,13.7,3.28,-8.22,-18.60,1900-06-20 00:00:00
3,190001,-82.40,27.38,1015.45,13.7,3.45,-0.38,-18.54,1900-06-20 06:00:00
4,190001,-82.46,28.31,1014.22,11.7,1.72,8.23,-4.21,1900-06-20 12:00:00
...,...,...,...,...,...,...,...,...,...
32824,198908,-77.88,40.72,998.40,12.5,83.26,-199.49,-198.54,1989-11-12 18:00:00
32825,198908,-80.75,42.25,1001.40,18.5,83.03,-293.51,-173.83,1989-11-13 00:00:00
32826,198908,-81.75,42.00,1001.48,14.8,73.58,-310.52,-142.11,1989-11-13 06:00:00
32827,198908,-82.50,42.00,1001.89,12.5,62.77,-296.47,-140.77,1989-11-13 12:00:00


In [9]:
Control_Codes

,ABC,Orig Code,New Code
0,A,0024,190001
1,A,0027,190002
2,A,0029,190003
3,A,0046,190004
4,A,0048,190005
...,...,...,...
940,C,1512,198904
941,C,1516,198905
942,C,1520,198906
943,C,1521,198907


In [10]:
# Open File
def Create_ET_DF(File, Codes_DF, Year_Diff, ABC):
    Data = open(File, 'r')
    Rows = []
#
# Organize Data
    for Line in Data:
        Rows.append(Line.strip())
#    print (Rows)
    Storm_Code_Orig = []
    Storm_Code_New = []
    Start_Time_Orig = []
    End_Time_Orig = []
    Start_Time_New = []
    End_Time_New = []
    Array = numpy.zeros((4, len(Rows)))
    for i in range(len(Rows)):
#        print (Rows[i][53:57])
        Storm_Code_Orig.append(str(Rows[i][0:4]))
        Array[0][i] = int(Rows[i][7:10]) * 6
        Array[1][i] = int(Rows[i][12])
        Array[2][i] = float(Rows[i][15:22])
        Array[3][i] = float(Rows[i][23:30])
        Start_Time_Orig.append(datetime.datetime(year=int(Rows[i][33:37]), month=int(Rows[i][39:41]), \
        day=int(Rows[i][43:45]), hour=int(Rows[i][47:49])))
        End_Time_Orig.append(datetime.datetime(year=int(Rows[i][53:57]), month=int(Rows[i][59:61]), \
        day=int(Rows[i][63:65]), hour=int(Rows[i][67:69])))
#
# Use New Storm Codes and Times
    for i in range(len(Storm_Code_Orig)):
        for m in range(len(Codes_DF)):
            if Storm_Code_Orig[i] == Codes_DF["Orig Code"][m] and ABC == Codes_DF["ABC"][m]:
                Storm_Code_New.append(Codes_DF["New Code"][m])
    for i in range(len(Start_Time_Orig)):
        Start_Time_New.append(Update_Year(Start_Time_Orig[i], Year_Diff))
        End_Time_New.append(Update_Year(End_Time_Orig[i], Year_Diff))
#
# Create DataFrame to Store Data
    ET_DF = pandas.DataFrame({"Code": Storm_Code_New, "Path Type": Array[1], \
    "Start Time": Start_Time_New, "End Time": End_Time_New, "ET Duration (hr)": Array[0], \
    "Start SLP": Array[2], "End SLP": Array[3]})
    return (ET_DF)

In [11]:
# Combine ET DataFrames
def ET_Combine_DF(ET_DF_A, ET_DF_B, ET_DF_C, Main_DF):
# Concatenate Other Variables
    Vars = ["ET Duration (hr)", "Path Type", "Start SLP", "End SLP"]
    Total_Length = len(ET_DF_A) + len(ET_DF_B) + len(ET_DF_C)
    Array = numpy.zeros((4,Total_Length))
    for n in range(4):
        Array[n] = numpy.concatenate((ET_DF_A[Vars[n]], ET_DF_B[Vars[n]], ET_DF_C[Vars[n]]))
    Codes = list(ET_DF_A["Code"]) + list(ET_DF_B["Code"]) + list(ET_DF_C["Code"])
    Start_Time = list(ET_DF_A["Start Time"]) + list(ET_DF_B["Start Time"]) + list(ET_DF_C["Start Time"])
    End_Time = list(ET_DF_A["End Time"]) + list(ET_DF_B["End Time"]) + list(ET_DF_C["End Time"])
    ET_DF = pandas.DataFrame({"Code": Codes, "Path Type": Array[1], "Start Time": Start_Time, "End Time": End_Time, \
    "ET Duration (hr)": Array[0], "Start SLP": Array[2], "End SLP": Array[3]})
    ET_DF = ET_Lat_Lon(Main_DF, ET_DF)
    ET_DF = ET_Distance(ET_DF)
    ET_DF = ET_Trop_Life(Main_DF, ET_DF)
    return (ET_DF)

In [12]:
# Add ET Transition Lat Lon Into ET DataFrame
def ET_Lat_Lon(Main_DF, ET_DF):
    Array = numpy.zeros((5, len(ET_DF)))
    for i in range(len(ET_DF)):
        Code = ET_DF["Code"][i]
        Start_Time = ET_DF["Start Time"][i]
        End_Time = ET_DF["End Time"][i]
        Storm = Main_DF[Main_DF["Code"] == Code]
        Array[0][i] = numpy.min(Storm["SLP(hPa)"])
        Array[1][i] = float(Storm[Storm["Time(Z)"] == Start_Time]["Lon"])
        Array[2][i] = float(Storm[Storm["Time(Z)"] == Start_Time]["Lat"])
        Array[3][i] = float(Storm[Storm["Time(Z)"] == End_Time]["Lon"])
        Array[4][i] = float(Storm[Storm["Time(Z)"] == End_Time]["Lat"])
    ET_DF["Min SLP"] = Array[0]
    ET_DF["Start Lon"] = Array[1]
    ET_DF["Start Lat"] = Array[2]
    ET_DF["End Lon"] = Array[3]
    ET_DF["End Lat"] = Array[4]
    return (ET_DF)

In [13]:
# Calculate Distance Between ET Start and End Points
def ET_Distance(ET_DF):
    Array = numpy.zeros(len(ET_DF))
    for i in range(len(ET_DF)):
        Start_Lat = ET_DF["Start Lat"][i] * numpy.pi / 180
        End_Lat = ET_DF["End Lat"][i] * numpy.pi / 180
        Start_Lon = ET_DF["Start Lon"][i] * numpy.pi / 180
        End_Lon = ET_DF["End Lon"][i] * numpy.pi / 180
        Lat_Diff = End_Lat - Start_Lat
        Lon_Diff = End_Lon - Start_Lon
        Earth_Rad = 6378
        Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
        numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
        Array[i] = Distance
    ET_DF["ET Distance (km)"] = Array
    return (ET_DF)

In [14]:
# Calculate Duration Between Storm Formation and Start of ET Transition
def ET_Trop_Life(Main_DF, ET_DF):
    Array = numpy.zeros(len(ET_DF))
    for i in range(len(ET_DF)):
        Code = ET_DF["Code"][i]
        Start_Time = ET_DF["Start Time"][i]
        Storm = Main_DF[Main_DF["Code"] == Code]
        Birth_Time = list(Storm["Time(Z)"])[0]
        Time_Diff = Start_Time - Birth_Time
        Hours = int(Time_Diff.total_seconds()) / 3600
        Array[i] = Hours
    ET_DF["Tropical Duration (hr)"] = Array
    return (ET_DF)

In [15]:
Control_A_ET_DF_Orig = Create_ET_DF(Diri+'etdetails_dtime900.002.txt', Control_Codes, Control_Year_Diffs[0], "A")
Control_B_ET_DF_Orig = Create_ET_DF(Diri+'etdetails_dtime900.002.txt', Control_Codes, Control_Year_Diffs[1], "B")
Control_C_ET_DF_Orig = Create_ET_DF(Diri+'etdetails_dtime900.002.txt', Control_Codes, Control_Year_Diffs[2], "C")
Control_ET_DF = ET_Combine_DF(Control_A_ET_DF_Orig, Control_B_ET_DF_Orig, Control_C_ET_DF_Orig, Control_DF)
Control_ET_DF

,Code,Path Type,Start Time,End Time,ET Duration (hr),Start SLP,End SLP,Min SLP,Start Lon,Start Lat,End Lon,End Lat,ET Distance (km),Tropical Duration (hr)
0,190003,1.0,1900-09-18 00:00:00,1900-09-20 12:00:00,60.0,952.06,979.68,934.83,-77.17,31.14,-62.68,47.61,2198.074309,198.0
1,190004,1.0,1900-11-05 12:00:00,1900-11-06 12:00:00,24.0,954.74,992.74,948.88,-87.08,27.00,-74.44,30.88,1301.648365,180.0
2,190005,1.0,1900-11-06 00:00:00,1900-11-08 00:00:00,48.0,996.02,998.22,988.17,-27.73,41.42,-9.00,45.00,1563.117613,126.0
3,190101,2.0,1901-05-24 12:00:00,1901-05-26 18:00:00,54.0,1000.16,999.92,962.99,-82.33,33.11,-80.25,46.75,1524.903429,180.0
4,190102,1.0,1901-05-28 12:00:00,1901-05-30 18:00:00,54.0,991.56,1008.78,980.89,-43.71,30.04,-34.75,24.00,1112.101864,132.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,198904,1.0,1989-09-09 18:00:00,1989-09-10 00:00:00,6.0,991.36,992.59,969.41,-69.25,44.34,-68.35,46.88,291.270597,288.0
434,198905,1.0,1989-09-20 18:00:00,1989-09-22 06:00:00,36.0,1001.48,993.73,993.73,-51.19,35.91,-41.30,43.86,1219.830294,42.0
435,198906,1.0,1989-10-12 12:00:00,1989-10-14 12:00:00,48.0,975.10,995.75,954.47,-50.44,25.10,-32.77,34.87,2007.931340,222.0
436,198907,1.0,1989-10-17 18:00:00,1989-10-18 18:00:00,24.0,980.42,1000.94,980.42,-80.16,31.55,-81.44,34.09,307.017016,126.0


In [16]:
# Cyclone Type Based on ET Transition Algorithm
def ET_Update(Main_DF, ET_DF):
    Codes = ET_DF["Code"]
    Final_DF = Main_DF[Main_DF["Code"].isin(Codes)].reset_index()
    ET_Type = []
    for i in range(len(Codes)):
        Storm_DF = Final_DF[Final_DF["Code"] == Codes[i]]
        Storm_ET_DF = ET_DF[ET_DF["Code"] == Codes[i]]
        Time = list(Storm_DF["Time(Z)"])
        Start_Time = list(Storm_ET_DF["Start Time"])[0]
        End_Time = list(Storm_ET_DF["End Time"])[0]
        for j in range(len(Time)):
            if Time[j] < Start_Time:
                ET_Type.append("Tropical")
            elif Time[j] < End_Time:
                ET_Type.append("Transition")
            else:
                ET_Type.append("Extratropical")
    Final_DF["Type"] = ET_Type
    Final_DF = Final_DF.drop("index", axis=1)
    return (Final_DF)

In [17]:
Control_DF_Final = ET_Update(Control_DF, Control_ET_DF)
Control_DF_Final

,Code,Lon,Lat,SLP(hPa),Winds(m/s),B,VLT,VUT,Time(Z),Type
0,190003,-32.20,12.39,1010.48,17.0,9.85,-25.24,17.78,1900-09-09 18:00:00,Tropical
1,190003,-33.72,12.32,1012.26,15.8,8.69,3.49,28.41,1900-09-10 00:00:00,Tropical
2,190003,-34.72,12.33,1010.48,18.1,8.78,4.94,31.79,1900-09-10 06:00:00,Tropical
3,190003,-36.53,12.93,1010.83,21.2,8.82,11.28,30.95,1900-09-10 12:00:00,Tropical
4,190003,-38.63,13.63,1008.41,21.8,8.53,29.05,24.23,1900-09-10 18:00:00,Tropical
...,...,...,...,...,...,...,...,...,...,...
18109,198908,-77.88,40.72,998.40,12.5,83.26,-199.49,-198.54,1989-11-12 18:00:00,Extratropical
18110,198908,-80.75,42.25,1001.40,18.5,83.03,-293.51,-173.83,1989-11-13 00:00:00,Extratropical
18111,198908,-81.75,42.00,1001.48,14.8,73.58,-310.52,-142.11,1989-11-13 06:00:00,Extratropical
18112,198908,-82.50,42.00,1001.89,12.5,62.77,-296.47,-140.77,1989-11-13 12:00:00,Extratropical
